# ASR Example - Basic Transcription Interface

Audio Speech Recognition with aisuite's unified API supporting OpenAI, Deepgram, and Google providers.

This example demonstrates basic transcription using the OpenAI format with different providers.

In [ ]:
import aisuite as ai
from aisuite.framework.message import TranscriptionResult
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())


# Set up client with provider configurations
client = ai.Client({
    "openai": {"api_key": os.getenv("OPENAI_API_KEY")},
    "deepgram": {"api_key": os.getenv("DEEPGRAM_API_KEY")},
    "google": {
        "project_id": os.getenv("GOOGLE_PROJECT_ID"),
        "region": os.getenv("GOOGLE_REGION"),
        "application_credentials": os.getenv("GOOGLE_APPLICATION_CREDENTIALS"),
    },
})

audio_file = "../aiplayground/speech.mp3"  # Replace with your audio file path

In [ ]:
# Basic transcription using kwargs (OpenAI format)
print("=== Basic Transcription ===")

try:
    result = client.audio.transcriptions.create(
        model="openai:whisper-1",
        file=audio_file,
        language="en"
    )
    if isinstance(result, TranscriptionResult):
        print(f"OpenAI: {result.text}")
    else:
        print("OpenAI: Got streaming result (not expected for basic call)")
except Exception as e:
    print(f"OpenAI error: {e}")

print("--------------------------------")

try:
    # Same kwargs work with other providers (auto-mapped)
    result = client.audio.transcriptions.create(
        model="deepgram:nova-2",
        file=audio_file,
        language="en",
        punctuate=True
    )
    if isinstance(result, TranscriptionResult):
        print(f"Deepgram: {result.text}")
    else:
        print("Deepgram: Got streaming result (not expected for basic call)")
except Exception as e:
    print(f"Deepgram error: {e}")

## Summary

This notebook demonstrates the unified ASR interface with:

1. **Basic transcription**: Using OpenAI-format kwargs that work across providers
2. **Provider compatibility**: Same interface works with OpenAI, Deepgram, and Google

### Environment Setup Required:

- **OpenAI**: `OPENAI_API_KEY`
- **Deepgram**: `DEEPGRAM_API_KEY`  
- **Google**: `GOOGLE_PROJECT_ID`, `GOOGLE_REGION`, `GOOGLE_APPLICATION_CREDENTIALS`

### Key Benefits:

- Write once, run on any provider
- Consistent error handling and response format
- Easy provider switching for testing and optimization